In [141]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [162]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import time

df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
labels = df.to_numpy()[:, 0]
scaler = StandardScaler()

df = df.drop(["label"], axis = 1)
dataset = df.to_numpy()

## Scale Dataset 
scaled_dataset = scaler.fit_transform(dataset)
## Run PCA on Dataset

start = time.perf_counter()
kmeans = MiniBatchKMeans(init='k-means++', n_init = 30, n_clusters = 120).fit(scaled_dataset)
#kmeans = GaussianMixture(n_components = 20).fit(scaled_dataset)
end = time.perf_counter()
print("Done in ", end - start, "seconds...")
"""
NUM_CLUSTERS = 15


distorsions = []
for i in range(1, NUM_CLUSTERS):
    kmeans = KMeans(init='k-means++', n_init = 12, n_clusters = i).fit(scaled_dataset)
    distorsions.append(kmeans.inertia_)
"""




In [143]:
"""
fig = plt.figure(figsize=(15, 5))
plt.plot(range(2,16), distorsions)
plt.grid(True)
plt.title('Elbow curve')
"""

In [163]:
import numpy as np 

print(kmeans.labels_)
print(labels)

dic = {}
for i in range(len(kmeans.labels_)):
    val = kmeans.labels_[i]
    if val in dic:
        dic[val][labels[i]] += 1
    else:
        dic[val] = [0 for i in range(26)]
        dic[val][labels[i]] += 1

for key in dic:
    dic[key] = np.argmax(dic[key])

print(dic)




In [164]:
import matplotlib.pyplot as plt 

test_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
test = test_df.to_numpy()

#labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
                      
def show_img(img_idx):
    plt.imshow(test[img_idx][1:].reshape(28, 28), cmap='gray')

def get_label(img_idx):
    return labels[test[img_idx][0]]

print(get_label(1))
show_img(1)

In [165]:
#Scale Test
#PCA on Test
scaled_test = scaler.fit_transform(test)


predictions_int = kmeans.predict(scaled_test[:, 1:])
predictions = []
for i in range(len(predictions_int)):
    predictions.append(dic[predictions_int[i]])

num_correct = 0
for i in range(test.shape[0]):
    if test[i][0] == predictions[i]:
        num_correct += 1
        
print("Accuracy is ", num_correct/test.shape[0] * 100, "%")





In [173]:
"""Example"""
img_idx = 10

show_img(img_idx)
prediction = kmeans.predict([scaled_test[img_idx, 1:]])
print(labels[dic[prediction[0]]])